In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the JSON files
with open('/content/drive/MyDrive/cap/bangla dataset/train_questions_save (1).json', 'r') as f:
    train_data = json.load(f)

with open('/content/drive/MyDrive/cap/bangla dataset/train_questions_save (1).json', 'r') as f:
    test_data = json.load(f)


In [ ]:
# Define image and question data
train_image_ids = []
train_questions = []
train_answers = []
test_image_ids = []
test_questions = []
test_answers = []

In [ ]:
for key in train_data:
    train_image_ids.append(train_data[key]['image_id'])
    train_questions.append(train_data[key]['question'])
    train_answers.append(train_data[key]['answer'])

for key in test_data:
    test_image_ids.append(test_data[key]['image_id'])
    test_questions.append(test_data[key]['question'])
    test_answers.append(test_data[key]['answer'])

In [ ]:
# Load and preprocess images
image_dir = '/content/drive/MyDrive/cap/bangla dataset/Images/'

In [ ]:
from PIL import Image

def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')
    img = img.resize((224, 224))
    img = img_to_array(img)
    img = img / 255.0
    return img

In [ ]:
train_images = [preprocess_image(image_dir + image_id + '.png') for image_id in train_image_ids]
train_images = np.array(train_images)

test_images = [preprocess_image(image_dir + image_id + '.png') for image_id in test_image_ids]
test_images = np.array(test_images)

In [ ]:
# Process questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_questions)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
train_question_sequences = tokenizer.texts_to_sequences(train_questions)
train_question_sequences = pad_sequences(train_question_sequences, padding='post')

In [ ]:
test_question_sequences = tokenizer.texts_to_sequences(test_questions)
test_question_sequences = pad_sequences(test_question_sequences, padding='post')

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D

# Define the VGG16 model
image_input = Input(shape=(224, 224, 3))
cnn_model = VGG16(weights='imagenet', include_top=False, input_tensor=image_input)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Add a Global Average Pooling layer after the VGG16 model
image_features = cnn_model.output
image_features = GlobalAveragePooling2D()(image_features)

In [ ]:
# Freeze the layers of the CNN model
for layer in cnn_model.layers:
    layer.trainable = False

In [ ]:
# Extract image features
train_image_features = cnn_model.predict(train_images)
test_image_features = cnn_model.predict(test_images)

31/31 [==============================] - 582s 19s/step


In [ ]:
# Define the LSTM model for processing questions
question_input = Input(shape=(None,))
embedding_layer = Embedding(vocab_size, 300, mask_zero=True)(question_input)
lstm_layer = LSTM(256)(embedding_layer)

# Combine image and question features
combined_features = Concatenate()([image_features, lstm_layer])
output = Dense(512, activation='relu')(combined_features)
output = Dense(vocab_size, activation='softmax')(output)

In [ ]:
# Create the final model
model = Model(inputs=[cnn_model.input, question_input], outputs=output)

In [ ]:
label_mapping = {'বানর বার': 0, 'বসা': 1, 'না': 2, 'মল': 3, 'ঠিক': 4, 'সাদা': 4, 'হ্যাঁ': 6, 'কুকুর': 7, 'বাদামী': 8, 'দুই': 9, 'চার': 10, 'পাঁচ': 11, 'সবুজ': 12, 'বাজানো': 13, 'বিড়াল': 14, 'নীল': 15, 'টেবিল': 16, 'ছেলে': 17, 'রৌদ্রোজ্জ্বল': 18, 'তিন': 19, 'কালো': 20, 'গাছ': 21, 'এক': 22, 'বেঞ্চ': 23, 'ধূসর': 24, 'হলুদ': 25, 'পাখি': 26, 'মেয়ে': 27, 'মদ': 28, 'মানুষ': 29, 'লাল': 30, 'বই': 31, 'হাড়': 32, 'সকার': 33, 'পাই': 34, 'লগ': 35, 'বেসবল': 36, 'ফুটবল': 37, 'কমলা': 38, 'পালঙ্ক': 39, 'দাঁড়িয়ে': 40, '0': 41, 'মহিলা': 42, 'খাবার': 43, 'মেঝে': 44, 'কাঠবিড়াল': 45, 'পাটি': 46, 'ঘুমাচ্ছেন': 47, 'আপেল': 48, 'বাইক': 49, 'উদ্ভিদ': 50, 'কম্বল': 51, 'ঘাস': 52, 'চেয়ার': 53, 'বাম': 54, 'কিছুই না': 55}

In [ ]:
train_label_encoded = [label_mapping[label] for label in train_answers]
train_label_encoded = np.array(train_label_encoded, dtype=np.int32)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit([train_images, train_question_sequences], np.array(train_label_encoded), epochs=15, batch_size=16)

Epoch 1/15
61/61 [==============================] - 593s 10s/step - loss: 3.1902 - accuracy: 0.3525
Epoch 2/15
61/61 [==============================] - 590s 10s/step - loss: 2.0924 - accuracy: 0.3699
Epoch 3/15
61/61 [==============================] - 589s 10s/step - loss: 1.7851 - accuracy: 0.4170
Epoch 4/15
61/61 [==============================] - 589s 10s/step - loss: 1.5232 - accuracy: 0.4836
Epoch 5/15
61/61 [==============================] - 586s 10s/step - loss: 1.3054 - accuracy: 0.5922
Epoch 6/15
61/61 [==============================] - 586s 10s/step - loss: 1.0991 - accuracy: 0.6650
Epoch 7/15
61/61 [==============================] - 585s 10s/step - loss: 0.9422 - accuracy: 0.7152
Epoch 8/15
61/61 [==============================] - 583s 10s/step - loss: 0.8169 - accuracy: 0.7357
Epoch 9/15
61/61 [==============================] - 584s 10s/step - loss: 0.7481 - accuracy: 0.7572
Epoch 10/15
12/61 [====>.........................] - ETA: 7:44 - loss: 0.6137 - accuracy: 0.8125

In [ ]:
# Plotting the loss and accuracy on the same graph
epochs = range(1, len(history.history['loss']) + 1)

plt.plot(epochs, history.history['loss'], label='Training Loss')
plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Training Loss and Accuracy over Epochs')
plt.legend()
plt.show()

In [ ]:
test_answers = [item['answer'] for _, item in test_data.items()]

test_label_encoded = [label_mapping[label] for label in test_answers]
test_label_encoded = np.array(test_label_encoded, dtype=np.int32)

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate([test_images, test_question_sequences], test_label_encoded)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)

In [ ]:
# Provide an image and question
input_image_path = '/content/drive/MyDrive/bangla dataset/Images/2.png'
input_question = 'কেউ কি সোফায় বসে আছে?'

# Set the maximum sequence length
max_question_length = 100

# Preprocess the input image
input_image = preprocess_image(input_image_path)
input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension

# Tokenize and preprocess the input question
input_question_sequence = tokenizer.texts_to_sequences([input_question])
input_question_sequence = pad_sequences(input_question_sequence, maxlen=max_question_length)

# Make prediction on the input
prediction = model.predict([input_image, input_question_sequence])
predicted_label = np.argmax(prediction)

# Decode the predicted label
label_mapping_reverse = {v: k for k, v in label_mapping.items()}
predicted_answer = label_mapping_reverse[predicted_label]

# Print the predicted answer
print("Predicted Answer:", predicted_answer)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have a function preprocess_image() and a tokenizer
# Also, you need to define the label_mapping dictionary and the model

# Function to check if a string is empty or consists of only spaces
def is_empty_or_spaces(s):
    return s.strip() == ""

input_image_path = input("Enter image path: ").strip()
input_question = input("Enter question: ").strip()

# Check if either image path or question is empty
if is_empty_or_spaces(input_image_path) and is_empty_or_spaces(input_question):
    print("Please enter an image path or a question.")
else:
    # Set the maximum sequence length
    max_question_length = 100

    # Preprocess the input image if not empty
    input_image = None
    if not is_empty_or_spaces(input_image_path):
        input_image = preprocess_image(input_image_path)
        input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension

    # Tokenize and preprocess the input question if not empty
    input_question_sequence = None
    if not is_empty_or_spaces(input_question):
        input_question_sequence = tokenizer.texts_to_sequences([input_question])
        input_question_sequence = pad_sequences(input_question_sequence, maxlen=max_question_length)

    if input_image is not None and input_question_sequence is not None:
        # Make prediction on the input
        prediction = model.predict([input_image, input_question_sequence])
        predicted_label = np.argmax(prediction)

        # Decode the predicted label
        label_mapping_reverse = {v: k for k, v in label_mapping.items()}
        predicted_answer = label_mapping_reverse[predicted_label]

        # Print the predicted answer
        print("Predicted Answer:", predicted_answer)
    else:
        print("Please provide either an image or a question.")